In [4]:
import pandas as pd
import numpy as np
import re

In [5]:
twitter_data = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', encoding = 'latin-1')

In [6]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [7]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Data Processing

In [10]:
twitter_data.shape

(1599999, 6)

In [11]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [12]:
column_names = ['target','id','date','flag','user','text']

In [13]:
twitter_data = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', 
                           encoding = 'latin-1', names = column_names)
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [14]:
twitter_data.shape

(1600000, 6)

In [15]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [16]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

## Converting the target column '4' into '1'

In [17]:
twitter_data.replace({'target':{4:1}},inplace = True)

In [18]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

### 0 ----> Negative Tweet
### 1 ----> Positive Tweet

In [19]:
port_stem = PorterStemmer()

In [20]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]','',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ''.join(stemmed_content)
    
    return stemmed_content

In [21]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)


In [22]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,isupsetthathecantupdatehisfacebookbytextingita...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichanidivedmanytimesfortheballmanagedtosave...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,mywholebodyfeelsitchyandlikeitsonfir
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclassnoitsnotbehavingatallimmadwhyam...


In [23]:
print(twitter_data['stemmed_content'])

0          switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1          isupsetthathecantupdatehisfacebookbytextingita...
2          kenichanidivedmanytimesfortheballmanagedtosave...
3                       mywholebodyfeelsitchyandlikeitsonfir
4          nationwideclassnoitsnotbehavingatallimmadwhyam...
                                 ...                        
1599995           justwokeuphavingnoschoolisthebestfeelingev
1599996    thewdbcomverycooltohearoldwaltinterviewshttpbl...
1599997         areyoureadyforyourmojomakeoveraskmefordetail
1599998     happythbirthdaytomybooofallltimetupacamarushakur
1599999    happycharitytuesdaythenspccsparkscharityspeaki...
Name: stemmed_content, Length: 1600000, dtype: object


In [24]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


## Separating the data and label

In [25]:
x = twitter_data['stemmed_content'].values
y = twitter_data['target'].values

In [26]:
print(x)

['switchfoothttptwitpiccomyzlawwwthatsabummeryoushouldagotdavidcarrofthirddaytodoitd'
 'isupsetthathecantupdatehisfacebookbytextingitandmightcryasaresultschooltodayalsoblah'
 'kenichanidivedmanytimesfortheballmanagedtosavetherestgooutofbound' ...
 'areyoureadyforyourmojomakeoveraskmefordetail'
 'happythbirthdaytomybooofallltimetupacamarushakur'
 'happycharitytuesdaythenspccsparkscharityspeakinguphh']


In [27]:
print(y)

[0 0 0 ... 1 1 1]


In [29]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2, stratify = y, random_state = 56)

In [30]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(1280000,)
(320000,)
(1280000,)
(320000,)


In [31]:
print(x_train)

['wowisitreallyfathersdaydamnmissinmydadihopehesherewatchinanyway'
 'rubybluedaisylikewisefollowfriday' 'tirednotreadyforworkthisweek' ...
 'jesatmhttptwitpiccomorkqiagre' 'daraveooohilikeyournewp'
 'icantcopewiththisheatnotwithahangoverineedmybedtobecoldrightnow']


In [32]:
print(x_test)

['lisagjivestruggledwiththatforeverasmuchasitellmyselfnottoiusuallydotrustfolksbuttimesoutofitburnsm'
 'feelingneglectedanduniformedbymyfamili'
 'twitterisntworkingonmycomputerlaaaameonstickam' ...
 'ryansitigetthattoooften' 'peopleaddmeonfacebooktalktomeonaim'
 'hourstogozzzzzzzzzzzzzzzzzzzzzzzsootir']


### Converting Text data into numeric

In [33]:
vectorizer = TfidfVectorizer()

x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [34]:
print(x_train)

  (0, 1227303)	1.0
  (1, 954832)	1.0
  (2, 1113387)	1.0
  (3, 605580)	1.0
  (4, 4956)	1.0
  (5, 946200)	1.0
  (6, 725876)	1.0
  (7, 847747)	1.0
  (8, 593753)	1.0
  (9, 610414)	1.0
  (10, 353623)	1.0
  (11, 1117179)	1.0
  (12, 302511)	1.0
  (13, 554544)	1.0
  (14, 540514)	1.0
  (15, 248846)	1.0
  (16, 721640)	1.0
  (17, 193124)	1.0
  (18, 753379)	1.0
  (19, 1094272)	1.0
  (20, 447988)	1.0
  (21, 549720)	1.0
  (22, 197329)	1.0
  (23, 994698)	1.0
  (24, 7383)	1.0
  :	:
  (1279975, 1069503)	1.0
  (1279976, 1191682)	1.0
  (1279977, 313976)	1.0
  (1279978, 1049580)	1.0
  (1279979, 222094)	1.0
  (1279980, 103934)	1.0
  (1279981, 158523)	1.0
  (1279982, 941260)	1.0
  (1279983, 437427)	1.0
  (1279984, 92141)	1.0
  (1279985, 855001)	1.0
  (1279986, 1231023)	1.0
  (1279987, 944528)	1.0
  (1279988, 1163873)	1.0
  (1279989, 957345)	1.0
  (1279990, 324297)	1.0
  (1279991, 756353)	1.0
  (1279992, 447606)	1.0
  (1279993, 633596)	1.0
  (1279994, 1054432)	1.0
  (1279995, 103636)	1.0
  (1279996, 206501)	

In [35]:
print(x_test)

  (46, 671322)	1.0
  (62, 1109802)	1.0
  (76, 455466)	1.0
  (77, 1061494)	1.0
  (104, 45700)	1.0
  (107, 1181823)	1.0
  (133, 874736)	1.0
  (166, 1043416)	1.0
  (170, 947518)	1.0
  (178, 834652)	1.0
  (186, 636574)	1.0
  (276, 507504)	1.0
  (299, 362402)	1.0
  (334, 173561)	1.0
  (390, 455749)	1.0
  (398, 451180)	1.0
  (410, 1147863)	1.0
  (413, 467580)	1.0
  (427, 801950)	1.0
  (434, 849406)	1.0
  (449, 262592)	1.0
  (481, 654920)	1.0
  (541, 473221)	1.0
  (601, 1222374)	1.0
  (614, 934886)	1.0
  :	:
  (318902, 37938)	1.0
  (318958, 1041248)	1.0
  (318994, 238586)	1.0
  (319015, 556649)	1.0
  (319058, 478714)	1.0
  (319120, 1177903)	1.0
  (319261, 330865)	1.0
  (319339, 561384)	1.0
  (319355, 1197780)	1.0
  (319357, 910465)	1.0
  (319390, 203410)	1.0
  (319441, 524305)	1.0
  (319519, 836309)	1.0
  (319549, 1101963)	1.0
  (319552, 1148355)	1.0
  (319597, 444535)	1.0
  (319615, 433278)	1.0
  (319648, 421196)	1.0
  (319657, 760544)	1.0
  (319738, 569004)	1.0
  (319742, 223088)	1.0
  (319

## Logistic Regression

In [53]:
model = LogisticRegression(max_iter = 1000)

In [54]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

## Model Evaluation

In [55]:
y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)

In [56]:
train_accuracy = accuracy_score(y_train,y_pred_train)
test_accuracy = accuracy_score(y_test,y_pred_test)

In [57]:
print('Training Accuracy : ', train_accuracy)
print('Test Accuracy :', test_accuracy)

Training Accuracy :  0.99813515625
Test Accuracy : 0.512453125


## saving the trained model

In [58]:
import pickle

In [59]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

## Using the saved model for future Predictions

In [60]:
## Loading the saved model 

loaded_model = pickle.load(open('trained_model.sav','rb'))

In [64]:
x_new = x_test[150]
print(y_test[150])

pred = model.predict(x_new)
print(pred)

if (pred[0] == 0):
    print('Neagtive Tweet')

else:
    print('Positive Tweet')

1
[1]
Positive Tweet
